# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


**Question 1**: What test is appropriate for this problem? Does CLT apply?

We can test this using A/B testing. CLT applies since we will be using a large number of permutations to test whether the data converges on a population proportion that is significantly different from the sample callback rate observed for black sounding names.

**Question 2**: What are the null and alternate hypotheses?

The null hypothesis is that race has no impact on the rate of callbacks for resumes. The alternate hypothesis is that black sounding names get a significantly lower callback rate than white sounding names.

**Question 3**: Compute margin of error, confidence interval, and p-value.

In [14]:
import numpy
black_candidates = data.loc[data['race'] == 'b']
white_candidates = data.loc[data['race'] == 'w']
black_callback = np.mean(black_candidates['call'])
white_callback = np.mean(white_candidates['call'])
total_callback = np.mean(data['call'])
print('Population mean:',total_callback)
print('Black candidates:',black_callback)
print('White candidates:', white_callback)

Population mean: 0.08049281686544418
Black candidates: 0.0644763857126236
White candidates: 0.09650924056768417


In [26]:
#Compute margin of error with 95% confidence (critical z of 1.96)
critical_z = 1.96
n = data.shape[0]
prop = total_callback
margin = critical_z * np.sqrt((prop * (1 - prop)) / n)
print('Margin of Error:', margin)

Margin of Error: 0.00764095631672


In [28]:
#Compute 95% confidence interval
conf_int = [prop - margin, prop + margin]
print('Confidence Interval:', conf_int)

Confidence Interval: [0.072851860548726013, 0.088133773182162353]


**Question 4**: Write a story describing the statistical significance in the context or the original problem.

In [22]:
#Draw 10,000 permutation replicates and compute the p-value
black = list(black_candidates['call'])
white = list(white_candidates['call'])
total = list(data['call'])
perm_replicates = np.empty(10000)
for i in range(len(perm_replicates)):
    permuted = np.random.permutation(total)
    perm_1 = permuted[:len(black)]
    perm_2 = permuted[len(black):]
    perm_replicates[i] = np.mean(perm_1)
p = np.sum(perm_replicates <= black_callback) / len(perm_replicates)
print('P-value =',p)

P-value = 0.0001


The probability of a random permutation replicate being less than or equal to the callback rate for resumes with black sounding names is 0.01%. We reject the null hypothesis that race has no impact on the callback rate. The callback rate for black sounding names is lower than the rate for white sounding names with an extremely high level of statistical significance.

**Question 5**: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis does not indicate that race/name is the most important factor in callback success. It simply shows that race/name is a factor, however there may be many factors at play that have not been analyzed yet. Improvements could be made to this analysis by conducting a logistic regression in which each feature's correlation coefficient and significance levels are tested in relation to the dependent variable (callback rate).